# CloudSim-Benchmark 实验结果可视化

本 Notebook 用于可视化 CloudSim-Benchmark 项目的实验结果，生成算法对比折线图。

## 功能特性

- 支持批处理模式和实时调度模式的结果可视化
- 绘制多个性能指标的对比折线图
- 支持单文件或多文件对比
- 自动识别 CSV 文件格式（单次运行或多次运行）


## 1. 导入必要的库


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# 设置中文字体支持
plt.rcParams['font.sans-serif'] = ['SimHei', 'Microsoft YaHei', 'Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False

# 设置图表样式
plt.style.use('seaborn-v0_8-darkgrid')

# 设置图表大小和 DPI
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['figure.dpi'] = 100

print("库导入成功！")


## 2. 配置参数


In [ ]:
# 结果文件目录
RESULTS_DIR = "results"

# 图表保存目录
FIGURES_DIR = "figures"

# 创建图表保存目录
Path(FIGURES_DIR).mkdir(exist_ok=True)

# 算法颜色映射（可根据需要调整）
ALGORITHM_COLORS = {
    'Random': '#FF6B6B',
    'PSO': '#4ECDC4',
    'WOA': '#45B7D1',
    'GWO': '#FFA07A',
    'HHO': '#98D8C8',
    'PSO-Realtime': '#4ECDC4',
    'WOA-Realtime': '#45B7D1',
    'GWO-Realtime': '#FFA07A',
    'HHO-Realtime': '#98D8C8',
    'MinLoad': '#95A5A6'
}

# 默认颜色（如果算法不在映射中）
DEFAULT_COLORS = plt.cm.tab10(np.linspace(0, 1, 10))

print(f"结果目录: {RESULTS_DIR}")
print(f"图表目录: {FIGURES_DIR}")


## 3. 读取结果文件


In [ ]:
def load_result_file(file_path):
    """
    读取单个结果 CSV 文件
    
    Parameters:
    -----------
    file_path : str
        CSV 文件路径
    
    Returns:
    -------
    pd.DataFrame
        包含结果的 DataFrame
    """
    try:
        df = pd.read_csv(file_path)
        print(f"✓ 成功读取: {os.path.basename(file_path)}")
        print(f"  算法数量: {len(df)}")
        print(f"  列数: {len(df.columns)}")
        return df
    except Exception as e:
        print(f"✗ 读取失败 {file_path}: {e}")
        return None


def list_result_files(mode='batch'):
    """
    列出指定模式的结果文件
    
    Parameters:
    -----------
    mode : str
        'batch' 或 'realtime'
    
    Returns:
    -------
    list
        文件路径列表
    """
    pattern = f"{RESULTS_DIR}/{mode}_comparison_*.csv"
    files = sorted(glob.glob(pattern))
    return files


# 列出所有结果文件
batch_files = list_result_files('batch')
realtime_files = list_result_files('realtime')

print(f"\n批处理模式文件数: {len(batch_files)}")
print(f"实时调度模式文件数: {len(realtime_files)}")

if batch_files:
    print("\n批处理模式文件:")
    for f in batch_files[-5:]:  # 显示最近5个
        print(f"  - {os.path.basename(f)}")

if realtime_files:
    print("\n实时调度模式文件:")
    for f in realtime_files[-5:]:  # 显示最近5个
        print(f"  - {os.path.basename(f)}")


In [ ]:
def plot_batch_comparison(file_path=None, save_fig=True):
    """
    绘制批处理模式算法对比折线图
    
    Parameters:
    -----------
    file_path : str, optional
        指定要绘制的文件路径，如果为 None 则使用最新的文件
    save_fig : bool
        是否保存图表
    """
    # 选择文件
    if file_path is None:
        files = list_result_files('batch')
        if not files:
            print("未找到批处理模式结果文件！")
            return
        file_path = files[-1]  # 使用最新的文件
    
    # 读取数据
    df = load_result_file(file_path)
    if df is None:
        return
    
    # 确定指标列（排除 Algorithm 列）
    metric_columns = [col for col in df.columns if col != 'Algorithm' and '_StdDev' not in col and '_Mean' not in col and col != 'Runs']
    
    # 创建子图
    n_metrics = len(metric_columns)
    n_cols = 2
    n_rows = (n_metrics + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5 * n_rows))
    fig.suptitle(f'批处理模式算法对比 - {os.path.basename(file_path)}', fontsize=16, fontweight='bold')
    
    # 如果只有一个子图，转换为数组
    if n_metrics == 1:
        axes = [axes]
    else:
        axes = axes.flatten()
    
    # 为每个指标绘制折线图
    for idx, metric in enumerate(metric_columns):
        ax = axes[idx]
        
        # 获取算法名称和对应的值
        algorithms = df['Algorithm'].values
        values = df[metric].values
        
        # 绘制折线图
        x_pos = np.arange(len(algorithms))
        
        # 为每个算法选择颜色
        colors = [ALGORITHM_COLORS.get(alg, DEFAULT_COLORS[i % len(DEFAULT_COLORS)]) 
                 for i, alg in enumerate(algorithms)]
        
        # 绘制折线和标记点
        ax.plot(x_pos, values, marker='o', linewidth=2, markersize=8, label=metric)
        
        # 为每个点设置颜色
        for i, (x, y, color) in enumerate(zip(x_pos, values, colors)):
            ax.scatter(x, y, c=[color], s=100, zorder=5)
        
        # 设置标签和标题
        ax.set_xticks(x_pos)
        ax.set_xticklabels(algorithms, rotation=45, ha='right')
        ax.set_ylabel(metric, fontsize=12)
        ax.set_title(f'{metric} 对比', fontsize=13, fontweight='bold')
        ax.grid(True, alpha=0.3)
        ax.legend()
    
    # 隐藏多余的子图
    for idx in range(n_metrics, len(axes)):
        axes[idx].axis('off')
    
    plt.tight_layout()
    
    # 保存图表
    if save_fig:
        filename = os.path.basename(file_path).replace('.csv', '_comparison.png')
        save_path = os.path.join(FIGURES_DIR, filename)
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"\n图表已保存到: {save_path}")
    
    plt.show()


# 绘制最新的批处理模式结果
if batch_files:
    plot_batch_comparison()
else:
    print("未找到批处理模式结果文件！")


In [ ]:
def plot_realtime_comparison(file_path=None, save_fig=True):
    """
    绘制实时调度模式算法对比折线图
    
    Parameters:
    -----------
    file_path : str, optional
        指定要绘制的文件路径，如果为 None 则使用最新的文件
    save_fig : bool
        是否保存图表
    """
    # 选择文件
    if file_path is None:
        files = list_result_files('realtime')
        if not files:
            print("未找到实时调度模式结果文件！")
            return
        file_path = files[-1]  # 使用最新的文件
    
    # 读取数据
    df = load_result_file(file_path)
    if df is None:
        return
    
    # 确定指标列（排除 Algorithm 列）
    metric_columns = [col for col in df.columns if col != 'Algorithm' and '_StdDev' not in col and '_Mean' not in col and col != 'Runs']
    
    # 创建子图
    n_metrics = len(metric_columns)
    n_cols = 2
    n_rows = (n_metrics + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5 * n_rows))
    fig.suptitle(f'实时调度模式算法对比 - {os.path.basename(file_path)}', fontsize=16, fontweight='bold')
    
    # 如果只有一个子图，转换为数组
    if n_metrics == 1:
        axes = [axes]
    else:
        axes = axes.flatten()
    
    # 为每个指标绘制折线图
    for idx, metric in enumerate(metric_columns):
        ax = axes[idx]
        
        # 获取算法名称和对应的值
        algorithms = df['Algorithm'].values
        values = df[metric].values
        
        # 绘制折线图
        x_pos = np.arange(len(algorithms))
        
        # 为每个算法选择颜色
        colors = [ALGORITHM_COLORS.get(alg, DEFAULT_COLORS[i % len(DEFAULT_COLORS)]) 
                 for i, alg in enumerate(algorithms)]
        
        # 绘制折线和标记点
        ax.plot(x_pos, values, marker='o', linewidth=2, markersize=8, label=metric)
        
        # 为每个点设置颜色
        for i, (x, y, color) in enumerate(zip(x_pos, values, colors)):
            ax.scatter(x, y, c=[color], s=100, zorder=5)
        
        # 设置标签和标题
        ax.set_xticks(x_pos)
        ax.set_xticklabels(algorithms, rotation=45, ha='right')
        ax.set_ylabel(metric, fontsize=12)
        ax.set_title(f'{metric} 对比', fontsize=13, fontweight='bold')
        ax.grid(True, alpha=0.3)
        ax.legend()
    
    # 隐藏多余的子图
    for idx in range(n_metrics, len(axes)):
        axes[idx].axis('off')
    
    plt.tight_layout()
    
    # 保存图表
    if save_fig:
        filename = os.path.basename(file_path).replace('.csv', '_comparison.png')
        save_path = os.path.join(FIGURES_DIR, filename)
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"\n图表已保存到: {save_path}")
    
    plt.show()


# 绘制最新的实时调度模式结果
if realtime_files:
    plot_realtime_comparison()
else:
    print("未找到实时调度模式结果文件！")


## 6. 自定义绘制（指定文件）

如果需要绘制特定的文件，可以取消注释下面的代码并修改文件路径。


In [ ]:
# 指定要绘制的文件（取消注释并修改文件路径）
# plot_batch_comparison(file_path='results/batch_comparison_20251222_201219.csv')
# plot_realtime_comparison(file_path='results/realtime_comparison_20251222_200426.csv')
